# Use case -  Harry potter Spell

## 1. By default, the spells in the Harry Potter series are primarily based on Latin or Latin-inspired words.

## 2. For our use case, let us say we want to change this behavior.

## 3. Let us say, we do not want latin words, but we will use english words and add some suffix to them make it sound like spell words.

## 4. We will use sentences with 2 words. To convert it in to spell word, we will make the first word to be second and the second to be first.

## 5. We will also add some suffix to the words to look like spell words.

## 6. For example
####      throw ball will become ballus throwum,
####      drink water will become waterade drinkum

## Steps for fine tuning

### Step 1. Prepare training data

### Step 2. Create fine tuning job and run training

### Step 3. Check status and metrics

### Step 4. use fine tuned model for prediction


## Step 1 Prepare Training Data

In [55]:
!cat spell_list.jsonl

{"messages": [{"role": "system", "content": "you are an assistant that will answer in harry potter spell format."}, {"role": "user", "content": "throw ball"}, {"role": "assistant", "content": "ballus throwum"}]}
{"messages": [{"role": "system", "content": "you are an assistant that will answer in harry potter spell format."}, {"role": "user", "content": "drink water"}, {"role": "assistant", "content": "waterade drinkum"}]}
{"messages": [{"role": "system", "content": "you are an assistant that will answer in harry potter spell format."}, {"role": "user", "content": "play piano"}, {"role": "assistant", "content": "pianus playum"}]}
{"messages": [{"role": "system", "content": "you are an assistant that will answer in harry potter spell format."}, {"role": "user", "content": "sing song"}, {"role": "assistant", "content": "songic singus"}]}
{"messages": [{"role": "system", "content": "you are an assistant that will answer in harry potter spell format."}, {"role": "user", "content": "eat foo

In [ ]:
from openai import OpenAI
import os

api_key = os.getenv("open_ai_secret_key")
client = OpenAI(api_key=api_key)

## Get Response before fine tuning

In [30]:
response = client.chat.completions.create(
  #model="gpt-3.5-turbo-0613",
    model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "you are an assistant that will answer in harry potter spell format."},
    {"role": "user", "content": "throw ball?"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='Accio Ball!', role='assistant', function_call=None, tool_calls=None)


## Check why llm gave the response 

In [47]:
response = client.chat.completions.create(
  #model="gpt-3.5-turbo-0613",
    model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "you are an helpful assistant who explains origin language of below"},
    {"role": "user", "content": "Accio Ball!"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='"Accio" is a Latin word that means "I summon" or "I call for". It is a command used within the wizarding world of Harry Potter to summon an object towards the wand-wielder. In this context, "Accio Ball" would be a spell used to summon a ball.', role='assistant', function_call=None, tool_calls=None)


## Step 2. Create fine tuning job and run training

In [ ]:
#upload training file
spell_file = client.files.create(
  file=open("spell_list.jsonl", "rb"),purpose="fine-tune"
)

In [31]:
client.fine_tuning.jobs.create(
  training_file=spell_file.id,
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-pnoDYapj3ivKfEWqMn3lxtZ5', created_at=1706465262, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-tTzcQbMEGIFBiKMGjDbIh6Zr', result_files=[], status='validating_files', trained_tokens=None, training_file='file-JPbH5fwE9oeHJSqrmMIkHmli', validation_file=None)

## Step 3. Check status and metrics

In [32]:
client.fine_tuning.jobs.list(limit=1)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-pnoDYapj3ivKfEWqMn3lxtZ5', created_at=1706465262, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=6, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-tTzcQbMEGIFBiKMGjDbIh6Zr', result_files=[], status='running', trained_tokens=None, training_file='file-JPbH5fwE9oeHJSqrmMIkHmli', validation_file=None)], object='list', has_more=True)

### Below is a screenshot from https://platform.openai.com/finetune

![open_ai_finetuning_job_metrics](metrics.png)


## find name of the fine tuned model

In [34]:

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-pnoDYapj3ivKfEWqMn3lxtZ5")

FineTuningJob(id='ftjob-pnoDYapj3ivKfEWqMn3lxtZ5', created_at=1706465262, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:personal::8m3x8R7u', finished_at=1706465628, hyperparameters=Hyperparameters(n_epochs=6, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-tTzcQbMEGIFBiKMGjDbIh6Zr', result_files=['file-7NRDM6NeupETHZG77tTDXoXA'], status='succeeded', trained_tokens=3192, training_file='file-JPbH5fwE9oeHJSqrmMIkHmli', validation_file=None)

### Step 4. use fine tuned model for prediction

## Sample 1 : Get response from untuned model

In [48]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0613",
  messages=[
    {"role": "system", "content": "you are an assistant that will answer in harry potter spell format."},
    {"role": "user", "content": "throw ball?"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='Engorgio Ball!', role='assistant', function_call=None, tool_calls=None)


## Sample 1 :Get response from fine tuned model

In [38]:
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0613:personal::8m3x8R7u",
  messages=[
    {"role": "system", "content": "you are an assistant that will answer in harry potter spell format."},
    {"role": "user", "content": "throw ball"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='ballus throwum', role='assistant', function_call=None, tool_calls=None)


## Sample 2 :Get response from fine tuned model

In [39]:
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0613:personal::8m3x8R7u",
  messages=[
    {"role": "system", "content": "you are an assistant that will answer in harry potter spell format."},
    {"role": "user", "content": "sing song"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='song sing', role='assistant', function_call=None, tool_calls=None)


## Sample 2 : Get response from untuned model

In [40]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0613",
  messages=[
    {"role": "system", "content": "you are an assistant that will answer in harry potter spell format."},
    {"role": "user", "content": "sing song"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='Sonorus! Let the sound be amplified, as I sing for you with joy and pride. La-la-la-la!', role='assistant', function_call=None, tool_calls=None)


## Sample 3 :Get response from fine tuned model

In [41]:
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0613:personal::8m3x8R7u",
  messages=[
    {"role": "system", "content": "you are an assistant that will answer in harry potter spell format."},
    {"role": "user", "content": "write essay"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content='essayus writeum', role='assistant', function_call=None, tool_calls=None)


## Sample 3 :Get response from untuned model

In [42]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0613",
  messages=[
    {"role": "system", "content": "you are an assistant that will answer in harry potter spell format."},
    {"role": "user", "content": "write essay"}
  ]
)
print(response.choices[0].message)

ChatCompletionMessage(content="Alohomora! Let the words flow and reveal the essay's topic. Lumos! Illuminate the path to a well-crafted introduction. Expecto Patronum! Summon the main ideas and arguments to defend your thesis. Incendio! Ignite the momentum with compelling evidence and examples. Wingardium Leviosa! Elevate your analysis to new heights. Riddikulus! Laugh away any potential weaknesses with solid counterarguments. Episkey! Mend any flaws in your logic or structure. Expecto Patronum! Conclude with a powerful summary that leaves a lasting impression. Finite Incantatem! End the essay with finesse and confidence.", role='assistant', function_call=None, tool_calls=None)
